In [1]:
from copy import deepcopy
from pathlib import Path
import os
import re
from copy import deepcopy
from pathlib import Path
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_parse import LlamaParse
from llama_index.core.schema import TextNode
from typing import Optional


In [2]:
import nest_asyncio

nest_asyncio.apply()

## Setting Up LlamaIndex and OpenAI Models

In this section, we initialize the `Settings` for LlamaIndex with the OpenAI embedding and language models. We use the `OpenAIEmbedding` model for embeddings and the `OpenAI` model for language processing.

In [81]:
OPENAI_API_KEY = os.getenv('TERTIARY_INFOTECH_API_KEY') 
LLAMA_API_KEY = os.getenv('LLAMA_CLOUD_API_KEY')

In [4]:
embed_model = OpenAIEmbedding(model="text-embedding-3-large", api_key=OPENAI_API_KEY)
llm = OpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

Settings.embed_model = embed_model
Settings.llm = llm

In [5]:
parser = LlamaParse(
    result_type="markdown",
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt-4o-mini",
)

In [6]:
print(f"Parsing slide deck...")
md_json_objs = parser.get_json_result(r"C:\Users\dljh1\Documents\courseware_autogen\Assessment\input\WSQ- Learner Guide Slides - Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini - v5.pdf")
md_json_list = md_json_objs[0]["pages"]

Parsing slide deck...
Started parsing the file under job_id ca20a00e-bea0-4d03-b623-d3591f78f92d


In [7]:
print(md_json_list[10]["md"])

# Final Assessment
- Written Assessment (SAQ) - 1 hr
- Practical Performance (PP) - 1 hr


In [8]:
print(md_json_list[1].keys())

dict_keys(['page', 'md', 'images', 'charts', 'items', 'status', 'links', 'triggeredAutoMode', 'structuredData', 'noStructuredContent', 'noTextContent'])


In [12]:
image_dicts = parser.get_images(md_json_objs, download_path="data_images")

> Image for page 1: [{'name': 'page_1.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 2: [{'name': 'page_2.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 3: [{'name': 'page_3.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 4: [{'name': 'page_4.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 5: [{'name': 'page_5.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 6: [{'name': 'page_6.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 7: [{'name': 'page_7.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 8: [{'name': 'page_8.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 9: [{'name': 'page_9.jpg', 'height': 0,

In [13]:
from llama_index.core.schema import TextNode
from typing import Optional

In [14]:
# get pages loaded through llamaparse
import re


def get_page_number(file_name):
    match = re.search(r"-page-(\d+)\.jpg$", str(file_name))
    if match:
        return int(match.group(1))
    return 0


def _get_sorted_image_files(image_dir):
    """Get image files sorted by page."""
    raw_files = [f for f in list(Path(image_dir).iterdir()) if f.is_file()]
    sorted_files = sorted(raw_files, key=get_page_number)
    return sorted_files

In [15]:
from copy import deepcopy
from pathlib import Path


# attach image metadata to the text nodes
def get_text_nodes(json_dicts, image_dir=None):
    """Split docs into nodes, by separator."""
    nodes = []

    image_files = _get_sorted_image_files(image_dir) if image_dir is not None else None
    md_texts = [d["md"] for d in json_dicts]

    for idx, md_text in enumerate(md_texts):
        chunk_metadata = {"page_num": idx + 1}
        if image_files is not None:
            image_file = image_files[idx]
            chunk_metadata["image_path"] = str(image_file)
        chunk_metadata["parsed_text_markdown"] = md_text
        node = TextNode(
            text="",
            metadata=chunk_metadata,
        )
        nodes.append(node)

    return nodes

In [16]:
# this will split into pages
text_nodes = get_text_nodes(md_json_list, image_dir="data_images")

In [17]:
print(text_nodes[10].get_content(metadata_mode="all"))

page_num: 11
image_path: data_images\ca20a00e-bea0-4d03-b623-d3591f78f92d-page_108.jpg
parsed_text_markdown: # Final Assessment
- Written Assessment (SAQ) - 1 hr
- Practical Performance (PP) - 1 hr


In [18]:
import os
from llama_index.core import (
    StorageContext,
    SummaryIndex,
    load_index_from_storage,
)

if not os.path.exists("storage_nodes_summary"):
    index = SummaryIndex(text_nodes)
    # save index to disk
    index.set_index_id("summary_index")
    index.storage_context.persist("./storage_nodes_summary")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage_nodes_summary")
    # load index
    index = load_index_from_storage(storage_context, index_id="summary_index")

In [67]:
from pydantic import BaseModel
from openai import OpenAI
from docx import Document
from typing import Dict, List
from llama_parse import LlamaParse
import nest_asyncio
import os
import json

# API access to llama-cloud
LLAMA_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")

# Using OpenAI API for embeddings/llms
OPENAI_API_KEY = os.getenv("TERTIARY_INFOTECH_API_KEY")

# Initialize the OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# class AssessmentMethod(BaseModel):
#     code: str
#     duration: str

# Define the Pydantic model for structured data extraction
class KnowledgeStatement(BaseModel):
    id: str
    text: str


class AbilityStatement(BaseModel):
    id: str
    text: str


class Topic(BaseModel):
    name: str
    subtopics: List[str]
    tsc_knowledges: List[KnowledgeStatement]
    tsc_abilities: List[AbilityStatement]


class LearningUnit(BaseModel):
    name: str
    topics: List[Topic]
    learning_outcome: str


class FacilitatorGuideExtraction(BaseModel):
    course_title: str
    tsc_proficiency_level: str
    learning_units: List[LearningUnit]

# Function to load FG document content using LlamaParse
def parse_with_llamaparse(file_path: str) -> str:
    """
    Parses the FG document using LlamaParse to preprocess its content.

    Args:
        file_path (str): Path to the document.

    Returns:
        str: Preprocessed content of the document as text.
    """

    parser = LlamaParse(
        api_key=LLAMA_API_KEY,
        result_type="markdown",
        show_progress=True,
        verbose=True,
        num_workers=8
    )

    result = parser.get_json_result(file_path)
    return result

# Function to load FG document content
def load_fg_document(file_path: str) -> str:
    """
    Reads the content of a Word document.

    Args:
        file_path (str): Path to the document.

    Returns:
        str: The document's text content.
    """
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

# Function to extract structured data from FG document
def extract_fg_data(preprocessed_content: str) -> FacilitatorGuideExtraction:
    """
    Extracts relevant information from the FG document using GPT and Pydantic.

    Args:
        preprocessed_content (str): Preprocessed text content of the FG document.

    Returns:
        FacilitatorGuideExtraction: Extracted and validated data.
    """
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    """You are an expert at structured data extraction. Extract the following details from the FG Document:
                    - Course Title
                    - TSC Proficiency Level
                    - Learning Units (LUs):
                        * Name of the Learning Unit
                        * Topics in the Learning Unit:
                            - Name of the Topic
                            - Description of the Topic (bullet points or sub-topics)
                            - Full Knowledge Statements associated with the topic, including their identifiers and text (e.g., K1: Range of AI applications)
                            - Full Ability Statements associated with the topic, including their identifiers and text (e.g., A1: Analyze algorithms in the AI applications)
                        * Learning Outcome (LO) for each Learning Unit
                    Return the output in a JSON format that matches the schema provided:
                    {
                        "course_title": "string",
                        "tsc_proficiency_level": "string",
                        "learning_units": [
                            {
                                "name": "string",
                                "topics": [
                                    {
                                        "name": "string",
                                        "subtopics": ["string"],
                                        "tsc_knowledges": [
                                            {"id": "string", "text": "string"}
                                        ],
                                        "tsc_abilities": [
                                            {"id": "string", "text": "string"}
                                        ]
                                    }
                                ],
                                "learning_outcome": "string"
                            }
                        ]
                    }
                    """
                ),
            },
            {"role": "user", "content": preprocessed_content},
        ],
        response_format=FacilitatorGuideExtraction,
    )

    return completion.choices[0].message.parsed

# Main Execution
if __name__ == "__main__":
    # Path to the FG document
    file_path = r"C:\Users\dljh1\Documents\courseware_autogen\Assessment\input\FG_TGS-2024042961_Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini_v2.docx"
    
    # Parse document content using LlamaParse
    try:
        parsed_content = parse_with_llamaparse(file_path)
        print("Preprocessed Content:")
        print(json.dumps(parsed_content, indent=4))
    except Exception as e:
        print(f"Error during LlamaParse parsing: {e}")
        exit(1)

    # Extract and validate structured data using OpenAI and Pydantic
    try:
        extracted_data = extract_fg_data(json.dumps(parsed_content))
        print("Structured Data:")
        print(extracted_data)
    except Exception as e:
        print(f"Error during OpenAI extraction: {e}")


Started parsing the file under job_id 07f16fe1-2a87-44d3-af3f-69ff8d9d25f2
Preprocessed Content:
[
    {
        "pages": [
            {
                "page": 1,
                "text": "                                    T\n                              Facilitator Guide\n                                         For\n\nDevelop Artificial Intelligence and Large Language Model (LLM) Applications\n                                with Google Gemini\n\n                           TGS Ref No: TGS-2024042961\n\n                                   Conducted by\n                         TERTIARY INFOTECH PTE. LTD\n\n                                 UEN: 201200696W\n\n                                     Version 2.0",
                "md": "# Facilitator Guide\n\n# For\n\n# Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini\n\nTGS Ref No: TGS-2024042961\n\n# Conducted by\n\nTERTIARY INFOTECH PTE. LTD\n\nUEN: 201200696W\n\nVersion 2.0",
             

In [75]:
from llama_index.llms.openai import OpenAI
from pydantic.v1 import BaseModel, Field
from typing import List
from IPython.display import display, Markdown

# System prompt tailored for content retrieval
system_prompt = """\
You are a content retrieval assistant tasked with retrieving educational topic content aligned with a given Knowledge Statement.

Your role:
1. Restrict your retrieval strictly to the specified topic provided in the query.
2. Retrieve and summarize the topic content that aligns with the provided Knowledge Statement.
3. If no specific content directly aligns with the Knowledge Statement, provide a general summary of the specified topic instead.
4. Identify and extract the exact inline segments from the provided documents that directly correspond to the content used to 
generate the given answer. The extracted segments must be verbatim snippets from the documents, ensuring a word-for-word match with the text 
in the provided documents.

Ensure that:
- (Important) Each segment is an exact match to a part of the document and is fully contained within the document text.
- The relevance of each segment to the generated answer is clear and directly supports the answer provided.
- (Important) If you didn't used the specific document don't mention it.
- If no relevant information is found for the Knowledge Statement, clearly state this and provide a general topic summary instead.

Restrictions:
- Do not include content from other topics or slides outside the specified topic.
- Each retrieved segment must explicitly belong to the given topic.
"""

llm = OpenAI(model="gpt-4o-mini", system_prompt=system_prompt)
# sllm = llm.as_structured_llm(output_cls=ReportOutput)

In [59]:
query_engine = index.as_query_engine(
    similarity_top_k=10,
    llm=llm,
    # response_mode="tree_summarize"
    response_mode="compact",
)

In [87]:
# Define a Pydantic model for the Knowledge Statement and its Retrieved Content
class KnowledgeStatementContent(BaseModel):
    knowledge_id: str = Field(..., description="The ID of the Knowledge Statement, e.g., K1, K2.")
    knowledge_statement: str = Field(..., description="The text of the Knowledge Statement.")
    topic_name: str = Field(..., description="The name of the topic associated with this Knowledge Statement.")
    retrieved_content: str = Field(..., description="The content retrieved for this Knowledge Statement.")

In [ ]:
# Initialize a list to store structured outputs
retrieved_data: List[KnowledgeStatementContent] = []

# Iterate through all learning units and topics to query for all Knowledge Statements
for learning_unit in extracted_data.learning_units:
    for topic in learning_unit.topics:
        for knowledge in topic.tsc_knowledges:
            knowledge_id = knowledge.id
            knowledge_statement = knowledge.text
            topic_name = topic.name

            print(f"\n--- Retrieving for Knowledge Statement: {knowledge_id}: {knowledge_statement}")
            
            # Query the index to retrieve topic content for this Knowledge Statement
            response = query_engine.query(
                f"Generate questions and answer content for the following topic: '{topic.name}' aligning to the Knowledge Statement: '{knowledge_statement}'."
            )
            retrieved_content = response.response

            # Add the structured data using Pydantic model
            try:
                knowledge_data = KnowledgeStatementContent(
                    knowledge_id=knowledge_id,
                    knowledge_statement=knowledge_statement,
                    topic_name=topic_name,
                    retrieved_content=retrieved_content
                )
                retrieved_data.append(knowledge_data)
            except Exception as e:
                print(f"Error adding structured data for {knowledge_id}: {e}")

# Print structured data for verification
for entry in retrieved_data:
    print(entry.json(indent=4))




--- Retrieving for Knowledge Statement: K1: Range of AI applications

--- Retrieving for Knowledge Statement: K6: Applicability of AI in the industry

--- Retrieving for Knowledge Statement: K4: Algorithm design and implementation

--- Retrieving for Knowledge Statement: K5: Methods of evaluating process improvements to the engineering processes using AI

--- Retrieving for Knowledge Statement: K3: Methods of evaluating effectiveness of AI applications

--- Retrieving for Knowledge Statement: K2: Concepts pertaining to performance effectiveness and analysis
{
    "knowledge_id": "K1",
    "knowledge_statement": "Range of AI applications",
    "topic_name": "Topic 1 Overview of Large Language Model (LLM)",
    "retrieved_content": "The content related to the topic \"Overview of Large Language Model (LLM)\" that aligns with the Knowledge Statement \"Range of AI applications\" includes the following:\n\n### Opportunities of LLM Applications\n- **Content Creation and Assistance**\n- **Cus

In [106]:
from pydantic import BaseModel, Field
from pydantic import BaseModel
from openai import OpenAI
from docx import Document
from typing import Dict, List
from llama_parse import LlamaParse
import nest_asyncio
import os
import json
OPENAI_API_KEY = os.getenv("TERTIARY_INFOTECH_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# Define the WSQ model for structured output
class WSQ(BaseModel):
    knowledge_statement: str = Field(..., description="The text of the Knowledge Statement.")
    scenario: str = Field(..., description="The realistic workplace scenario.")
    question: str = Field(..., description="The question based on the scenario.")
    answer: str = Field(..., description="The concise answer to the question.")

# Function to generate WSQ using OpenAI
def generate_wsq_openai(knowledge_statement: str, topic_name: str, retrieved_content: str) -> WSQ:
    """
    Generate a Workplace Scenario Question (WSQ) using OpenAI.

    Args:
        knowledge_statement (str): The Knowledge Statement.
        topic_name (str): The name of the topic.
        retrieved_content (str): The content retrieved for the Knowledge Statement.

    Returns:
        WSQ: A structured WSQ object.
    """
    messages = [
        {
            "role": "system",
            "content": "You are an experienced instructional designer tasked with creating Workplace Scenario Questions (WSQs). Your output must include a Knowledge Statement, Scenario, Question, and Answer in JSON format."
        },
        {
            "role": "user",
            "content": (
                f"Generate a WSQ for the following inputs:\n\n"
                f"Knowledge Statement: {knowledge_statement}\n"
                f"Topic Name: {topic_name}\n"
                f"Retrieved Content: {retrieved_content}\n\n"
                f"Ensure the response adheres to this schema: {{'knowledge_statement': 'string', 'scenario': 'string', 'question': 'string', 'answer': 'string'}}."
            )
        }
    ]

    # Call OpenAI's structured response API
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=messages,
        response_format=WSQ
    )

    try:
        wsq_data = response.choices[0].message.parsed
        return wsq_data
    except Exception as e:
        print(f"Error generating WSQ: {e}")
        return None


In [107]:
# Generate WSQs for each Knowledge Statement in retrieved_data
wsq_results = []

for entry in retrieved_data:
    # Access attributes of the Pydantic model
    knowledge_statement = entry.knowledge_statement
    topic_name = entry.topic_name
    retrieved_content = entry.retrieved_content

    # Generate WSQ
    wsq = generate_wsq_openai(knowledge_statement, topic_name, retrieved_content)
    if wsq:
        print(f"\n--- WSQ for {knowledge_statement} ---\n")
        print(f"Scenario: {wsq.scenario}\n")
        print(f"Question: {wsq.question}\n")
        print(f"Answer: {wsq.answer}\n")
        wsq_results.append(wsq)

# Save WSQs to a file for further use
with open("wsq_results.json", "w") as f:
    json.dump([wsq.dict() for wsq in wsq_results], f, indent=4)


--- WSQ for Range of AI applications ---

Scenario: A team meeting is being held to discuss the potential applications of Large Language Models (LLMs) in our company. The marketing department is particularly interested in how LLMs can assist in content creation and customer support, while the IT department is focused on coding assistance and development. Additionally, the HR department wants to explore how LLMs can help in legal compliance and employee education.

Question: What specific applications of Large Language Models should each department consider based on their needs during the team meeting?

Answer: The marketing department should consider content creation and customer support applications, the IT department should focus on coding and development tools, and the HR department should explore applications related to legal compliance and educational tools.


--- WSQ for Applicability of AI in the industry ---

Scenario: As a project manager in a tech company, you are exploring 

In [112]:
print(wsq_results)

[WSQ(knowledge_statement='Range of AI applications', scenario='A team meeting is being held to discuss the potential applications of Large Language Models (LLMs) in our company. The marketing department is particularly interested in how LLMs can assist in content creation and customer support, while the IT department is focused on coding assistance and development. Additionally, the HR department wants to explore how LLMs can help in legal compliance and employee education.', question='What specific applications of Large Language Models should each department consider based on their needs during the team meeting?', answer='The marketing department should consider content creation and customer support applications, the IT department should focus on coding and development tools, and the HR department should explore applications related to legal compliance and educational tools.'), WSQ(knowledge_statement='Applicability of AI in the industry', scenario='As a project manager in a tech comp

In [111]:
for wsq in wsq_results:
    print(wsq.dict())

{'knowledge_statement': 'Range of AI applications', 'scenario': 'A team meeting is being held to discuss the potential applications of Large Language Models (LLMs) in our company. The marketing department is particularly interested in how LLMs can assist in content creation and customer support, while the IT department is focused on coding assistance and development. Additionally, the HR department wants to explore how LLMs can help in legal compliance and employee education.', 'question': 'What specific applications of Large Language Models should each department consider based on their needs during the team meeting?', 'answer': 'The marketing department should consider content creation and customer support applications, the IT department should focus on coding and development tools, and the HR department should explore applications related to legal compliance and educational tools.'}
{'knowledge_statement': 'Applicability of AI in the industry', 'scenario': 'As a project manager in a

In [113]:
context = {
    "course_title": extracted_data.course_title,  # e.g., extracted_data.course_title
    "type": "Written Assessment (SAQ)",
    "code": "SAQ",
    "duration": "1 hr",
    # The scenario field at the top level can be a general overview or introduction.
    # Since each WSQ also has its own scenario, you can choose to put a general scenario here:
    "scenario": "Below are scenario-based questions derived from the course content and Knowledge Statements.",
    "questions": []
}

In [114]:
# Populate the questions from wsq_results
for wsq in wsq_results:
    # Each question now includes the WSQ fields directly
    context["questions"].append({
        "knowledge_statement": wsq.knowledge_statement,
        "scenario": wsq.scenario,
        "question_statement": wsq.question,
        "answer": wsq.answer
    })

In [ ]:
from docxtpl import DocxTemplate
import os

def generate_documents(context: dict, type: int, output_dir: str) -> dict:
    """
    Generate the question paper and answer paper for the given context and type.

    Parameters:
    - context (dict): The data for the assessment (course title, type, questions, etc.).
    - type (int): The assessment type (1 for Ability-based, 2 for Knowledge-based).
    - output_dir (str): Directory where the generated documents will be saved.

    Returns:
    - dict: Paths to the generated documents (question and answer papers).
    """
    # Define template paths
    TEMPLATES = {
        1: {
            "ANSWER": r"C:\Users\dljh1\Documents\courseware_autogen\Assessment\Templates\(Template) Answer to A Assessment - Course Title - v1.docx",
            "QUESTION": r"C:\Users\dljh1\Documents\courseware_autogen\Assessment\Templates\(Template) A Assessment - Course Title - v1.docx"
        },
        2: {
            "ANSWER": r"C:\Users\dljh1\Documents\courseware_autogen\Assessment\Templates\(Template) Answer to K Assessment - Course Title - v1.docx",
            "QUESTION": r"C:\Users\dljh1\Documents\courseware_autogen\Assessment\Templates\(Template) WA (SAQ) - Course Title - v1.docx"
        },
    }

    # Validate type
    if type not in TEMPLATES:
        raise ValueError("Invalid type. Must be 1 (Ability-based) or 2 (Knowledge-based).")

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load templates
    answer_template_path = TEMPLATES[type]["ANSWER"]
    question_template_path = TEMPLATES[type]["QUESTION"]
    answer_doc = DocxTemplate(answer_template_path)
    question_doc = DocxTemplate(question_template_path)

    # Prepare context for the question paper by creating a copy of the context without answers
    question_context = {
        **context,
        "questions": [
            {
                **question,
                "answer": None,  # Remove answers for the question document
                "knowledge_statement_id": question["knowledge_statement"].split(":")[0]  # Extract ID
            }
            for question in context.get("questions", [])
        ]
    }

    # Render both templates
    answer_doc.render(context)  # Render with answers
    question_doc.render(question_context)  # Render without answers

    # Save the documents to the output directory
    files = {
        "ANSWER": os.path.join(output_dir, f"Answers to WA(SAQ) - {context['course_title']} - v1.docx"),
        "QUESTION": os.path.join(output_dir, f"WA(SAQ) - {context['course_title']} - v1.docx")
    }
    answer_doc.save(files["ANSWER"])
    question_doc.save(files["QUESTION"])

    return files  # Return paths to the generated documents

In [131]:
files = generate_documents(context, type=2, output_dir="output" )  # 'type=2' as per your code for knowledge-based assessment
print("Documents generated:", files)

{'course_title': 'Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini', 'type': 'Written Assessment (SAQ)', 'code': 'SAQ', 'duration': '1 hr', 'scenario': 'Below are scenario-based questions derived from the course content and Knowledge Statements.', 'questions': [{'knowledge_statement': 'Range of AI applications', 'scenario': 'A team meeting is being held to discuss the potential applications of Large Language Models (LLMs) in our company. The marketing department is particularly interested in how LLMs can assist in content creation and customer support, while the IT department is focused on coding assistance and development. Additionally, the HR department wants to explore how LLMs can help in legal compliance and employee education.', 'question_statement': 'What specific applications of Large Language Models should each department consider based on their needs during the team meeting?', 'knowledge_statement_id': 'Range of AI applications'},